In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
import math

In [ ]:
class SLDS(object):
    def __init__(self, K = None, T=None, transition_dis = None, init_prob = None, transition_con = None, emission = None, Gamma = None, Sigma = None, u0 = None, V0 = None, y=None):

        if(transition_con is None or transition_dis is None or emission is None):
            raise ValueError("Set proper system dynamics.")

        self.y = y # T x N
        self.K = K # dimension of hidden discrete states
        self.M = transition_con.shape[0] # dimension of hidden continuous states
        self.T = T # number of observations
        self.N = self.y.shape[1] # dimension of the observations

        self.transition_dis = transition_dis # the discrete variable transition probability matrix, K x K
        self.init_prob = init_prob # the initial probability of the discrete variable, K x 1
        self.emission = emission # C is the emission probability matrix, K x N x M
        self.transition_con = transition_con # the continuous variable transition probability matrix, K x M x M 
        
        self.Gamma = np.eye(self.M) if Gamma is None else Gamma # Gamma is the covariance matrix of noise term added to the hidden state transition, M x M
        self.Sigma = np.eye(self.M) if Sigma is None else Sigma # Sigma is the covariance matrix of noise term added to the emission, N x N

        self.h = np.zeros((self.T,self.K)) # variational parameter h
        self.q = np.zeros((self.T,self.K)) # variational parameter q
        
        self.P = np.zeros((self.T, self.M, self.M))
        self.P[:,:,] = np.eye(self.M)  # P is an intermediate variable during inference, N x M x M
        self.u = np.zeros((self.T, self.M)) # T x M x 1
        self.V = np.zeros((self.T, self.M, self.M)) # T x M x M
        self.K = np.zeros((self.T, self.M, self.N)) # T x M x N
        self.c = np.zeros((self.T)) # T x 1

        # for backward passing
        self.u_hat = np.zeros((self.T, self.M)) # T x M x 1
        self.V_hat = np.zeros((self.T, self.M, self.M)) # T x M x M
        self.J = np.zeros((self.T, self.M, self.M)) # T x M x M

        self.u0 = u0 # u0 is the initial estimate of the mean of z1, M x 1
        self.V0 = V0 # V0 is the initial estimate of the variance of z1, M x M
        
    
    def update_h(self):        
        forward, backward = self.forward_backward(self.init_prob, self.q, self.transition_dis, self.y)
        a = np.zeros((self.T,self.K))
        for i in range(self.T):
            for j in range(self.K):
                a[i,j] = forward[i,j]*backward[i,j]
            temp = np.sum(a[i,:])
            a[i,:] = a[i,:]/temp
        self.h = self.a

    def forward_backward(self,init_prob,emission,transition,data):
        forward_hat = np.zeros((self.T,self.K))
        backward_hat = np.zeros((self.T,self.K))
        scale_factors = np.zeros((self.T))

        forward_hat[0,:] = init_prob * emission[:,data[0]]
        scale_factors[0] = np.sum(forward_hat[0,:])
        forward_hat[0,:] = forward_hat[0,:]/scale_factors[0]
        
        for t in range(self.T-1):
            temp = np.matmul(forward_hat[t,:] ,transition) * emission[:,data[t+1]]
            scale_factors[t+1] = np.sum(temp)
            forward_hat[t+1,:] = temp/scale_factors[t+1]
            # print(f'temp is {temp} and the scale factor is {scale_factors[t+1]} and the forward_hat is {forward_hat[t+1]}')

        backward_hat[-1,:] = scale_factors[-1]
        for t in reversed(range(self.T-1)):
            temp = np.matmul(backward_hat[t+1,:]*emission[:,data[t+1]],transition.T)
            backward_hat[t,:] = temp/scale_factors[t]
        return forward_hat, backward_hat

        # the end of forward-backward algorithm

    def update_q(self):
        self.Xt = np.zeros((self.T,self.K,self.M))
        self.XtXt = np.zeros((self.T,self.K,self.M,self.M))
        self.Xt = self.u_hat
        for i in range(self.T):
            for j in range(self.K):
                self.q[i,j] = np.exp(-1/2*np.matmul(np.matmul(self.y[i,:].T,np.linalg.inv(self.Sigma)),self.y[i,:])+
                                     np.matmul(np.matmul(np.matmul(self.y[i,:].T, np.linalg.inv(self.Sigma)),self.emission[j]), self.Xt[i,j])-
                                               1/2* np.linalg.inv(self.emission[j]).T @ np.linalg.inv(self.Sigma) @ self.emission[j] @ self.XtXt[i,j])
                                               
        # the end of update q

    def kalman_filtering(self):
    
        S_temp = np.matmul(np.matmul(self.C, self.V0), self.C.T) + self.Sigma
        Q_temp = np.matmul(self.C, self.u0)
        I = np.eye(self.M)
        self.V[0] = np.matmul((I - np.matmul(np.matmul(np.matmul(self.V0, self.C.T), np.linalg.inv(S_temp)), self.C)), self.V0)
        self.P[0] = np.matmul(np.matmul(self.A, self.V[0]), self.A.T) + self.Gamma
        self.K[0] = np.matmul(np.matmul(self.P[0], self.C.T), np.linalg.inv(np.matmul(np.matmul(self.C, self.P[0]), self.C.T) + self.Sigma))
        
        self.u[0] = self.u0 + np.matmul(self.K[0], self.y[0] - Q_temp)
        # print(self.u0,self.u[0],self.P[0],self.K[0],self.V[0])
        self.c[0] = multivariate_normal.pdf(self.y[0], Q_temp, S_temp)
        for i in range(1,self.T,1):
            I = np.eye(self.M)
            Q_temp = np.matmul(np.matmul(self.C, self.A), self.u[i-1])
            
            self.V[i] = np.matmul((I - np.matmul(self.K[i-1], self.C)), self.P[i-1])
            self.P[i] = np.matmul(np.matmul(self.A, self.V[i]), self.A.T) + self.Gamma
            S_temp = np.matmul(np.matmul(self.C, self.P[i]), self.C.T) + self.Sigma

            self.K[i] = np.matmul(np.matmul(self.P[i], self.C.T), np.linalg.inv(S_temp))

            self.u[i] = np.matmul(self.A, self.u[i-1]) + np.matmul(self.K[i-1], self.y[i] - Q_temp)

            self.c[i] = multivariate_normal.pdf(self.y[i], Q_temp, S_temp)


    def kalman_smoothing(self):
        self.u_hat[-1] = self.u[-1]
        self.V_hat[-1] = self.V[-1]

        for i in range(self.T-2,-1,-1):

            self.J[i] = np.matmul(np.matmul(self.V[i], self.A.T), np.linalg.inv(self.P[i]))
            self.u_hat[i] = self.u[i] + np.matmul(self.J[i], self.u_hat[i+1] - np.matmul(self.A, self.u[i]))
            self.V_hat[i] = self.V[i] + np.matmul(np.matmul(self.J[i], self.V_hat[i+1] - self.P[i]), self.J[i].T)
    
    def learning(self,M,N):
        self.u0 = self.u_hat[0]
        self.V0 = self.V_hat[0] + np.outer(self.u_hat[0], self.u_hat[0].T) - np.outer(self.u_hat[0], self.u_hat[0].T)

        # E[z[n]] : M x 1
        # E[z[n]z[n-1].T] : M x M
        # E[z[n]z[n].T] : M x M

        sub_1 = np.zeros((M,M))
        sub_2 = np.zeros((M,M))
        sub_3 = np.zeros((M,M))
        sub_4 = np.zeros((M,M))

        sub_5 = np.zeros((N,M))
        sub_6 = np.zeros((M,M))
        sub_7 = np.zeros((N,N))
        sub_8 = np.zeros((M,N))

        for i in range(1,self.T,1):
            sub_1 += np.matmul(self.V_hat[i],self.J[i-1].T) + np.outer(self.u_hat[i],self.u_hat[i-1].T) # z[n]z[n-1]
            
            sub_2 += self.V_hat[i-1] + np.outer(self.u_hat[i-1], self.u_hat[i-1].T)

            sub_3 += self.V_hat[i] + np.outer(self.u_hat[i], self.u_hat[i].T) # z[n]z[n]
            sub_4 += (np.matmul(self.V_hat[i],self.J[i-1].T) + np.outer(self.u_hat[i],self.u_hat[i-1].T)).T #z[n-1]z[n]

        for i in range(self.T):
            sub_5 += np.outer(self.y[i], self.u_hat[i].T) # x[n] * E[z[n]].T
            sub_6 += self.V_hat[i] + np.outer(self.u_hat[i], self.u_hat[i].T) # z[n]z[n]
            sub_7 += np.outer(self.y[i], self.y[i].T) # x[n]x[n]
            sub_8 += np.outer(self.u_hat[i], self.y[i].T) #E[z[n]] * x[n].T 

        self.transition_con = np.matmul(sub_1, np.linalg.inv(sub_2))

        self.Gamma = 1/(self.T-1) * (sub_3 - np.matmul(self.A,sub_4) - np.matmul(sub_1,self.A.T) + np.matmul(np.matmul(self.A,sub_2),self.A.T))
        
        self.emission = np.matmul(sub_5, np.linalg.inv(sub_6))
        self.Sigma = 1/self.T * (sub_7 - np.matmul(self.C,sub_8) - np.matmul(sub_5,self.C.T) + np.matmul(np.matmul(self.C,sub_6),self.C.T))


In [ ]:
def generate_examples(A = None, C=None, Gamma=None,Sigma=None,u0=None,V0=None,M=None,N=None,T=None,z0=None):
 
    z = np.zeros((T,M))
    x = np.zeros((T,N))
    if z0 is not None:
        z[0]=z0
    else:
        z[0] = np.random.multivariate_normal(u0,V0)
    x[0] = np.random.multivariate_normal(np.matmul(C,z[0]),Sigma)
    for t in range(1,T,1):
        z[t] = np.random.multivariate_normal(np.matmul(A,z[t-1]),Gamma)
        x[t] = np.random.multivariate_normal(np.matmul(C,z[t]),Sigma)
    return z,x


In [ ]:
def generate_examples(N):

    np.random.seed(1000)

    # rotate pi / 6 radian in any axis
    A = np.matmul(
        np.matmul(
            np.array([
                [1.0,0.0,0.0],
                [0.0,math.cos(math.pi/6),math.sin(math.pi/6)],
                [0.0,-1.0*math.sin(math.pi/6),math.cos(math.pi/6)]
            ]),
            np.array([
                [math.cos(math.pi/6),0.0,-1.0*math.sin(math.pi/6)],
                [0.0,1.0,0.0],
                [math.sin(math.pi/6),0.0,math.cos(math.pi/6)]
            ])),
        np.array([
            [math.cos(math.pi/6),math.sin(math.pi/6),0.0],
            [-1.0*math.sin(math.pi/6),math.cos(math.pi/6),0.0],
            [0.0,0.0,1.0]
        ])
    )

    Gamma = np.array([
        [1.5, 0.1, 0.0],
        [0.1, 2.0, 0.3],
        [0.0, 0.3, 1.0]
    ])

    Z = np.array([[23.0, 24.0, 25.0]])
    for n in range(N):
        z_prev = Z[len(Z) - 1]
        mean = np.matmul(A, z_prev)
        z_post = np.random.multivariate_normal(
            mean=mean,
            cov=Gamma,
            size=1)
        Z = np.vstack((Z, z_post))

    C = np.array([
        [1.0, 1.0, 0.0],
        [0.0, 1.0, 1.0],
    ])

    Sigma = np.array([
        [1.0,0.2],
        [0.2,2.0],
    ])

    X = np.empty((0,2))
    for z_n in Z:
        mean = np.matmul(C, z_n)
        x_n = np.random.multivariate_normal(
            mean=mean,
            cov=Sigma,
            size=1)
        X = np.vstack((X, x_n))
    return Z, X

In [ ]:
def main():
	
	n_states = 3 # M
	n_obs = 2 # N
	n_time = 2000 # T
	p_old = -10000
	tol = 0.0001
	max_iter = 500

	# z: T x M
	# x : T x N
	# A = np.array([[0.9, 0.1],[0.5,0.5]])
	# C = np.array([[1, 0],[0.2, 0.8]])
	# Gamma = np.array([[0.1, 0.1], [0.1, 0.1]])
	# Sigma = np.array([[0.5,0.5],[0.5,0.5]])

	A = np.array([[0.75, 0.433, -0.5],[-0.217, 0.875, 0.433],[0.625, -0.217, 0.75]])
	Gamma = np.array([[1.5, 0.1, 0.0], [0.1, 2.0, 0.3], [0.0, 0.3, 1.0]])
	C = np.array([[1.0,1.0,0.0],[0.0,1.0,1.0]])
	Sigma = np.array([[1.0,0.2], [0.2,2.0]])

	u0 = np.array([1,2])
	V0 = np.array([[0.1,0.3],[0.3,0.1]])
	z0 = np.array([[23.0, 24.0, 25.0]])
	# A_init = np.array([[0.5, 0.5],[0.5,0.5]])
	# C_init = np.array([[0.5, 0.5],[0.5, 0.5]])
	# Gamma_init = np.array([[0.5, 0.9], [0.9, 4.5]])
	# Sigma_init = np.array([[0.5, 0.9], [0.9, 2.5]])
	# u0_init = np.array([1,2])
	# V0_init = np.array([[0.2,0.5],[0.5,0.4]])

	A_init = np.array([[1.0, 1.1, 1.2],[1.3, 1.4, 1.5],[1.6, 1.7, 1.8]])
	C_init = np.array([[1.0,1.0,1.0], [1.0, 1.0,1.0]])
	Gamma_init = np.array([[1.0, 0.5, 0.5], [0.5,1.0, 0.5],[0.5, 0.5, 1.0]])
	Sigma_init = np.array([[1.0,0.5], [0.5,1.0]])
	u0_init = np.array([10.0,10.0,10.0])
	V0_init = np.array([[1.0, 0.5, 0.5], [0.5,1.0, 0.5],[0.5, 0.5, 1.0]])


	# z,x = generate_examples(A = A, C = C, Gamma = Gamma, Sigma = Sigma, z0=z0,M=n_states,N=n_obs,T=n_time)
	z,x = generate_examples(n_time)

	kf = SLDS(A = A_init, C = C_init, Gamma = Gamma_init, Sigma = Sigma_init, u0=u0_init, V0=V0_init,x=x, T=n_time)
	
	for ite in range(max_iter):
		for t in range(1,kf.T,1):
			kf.forward(t)
		kf.u_hat[-1] = kf.u[-1]
		kf.V_hat[-1] = kf.V[-1]

		for t in range(kf.T-2,-1,-1):
			kf.backward(t)
		kf.learning(z.shape[1],x.shape[1])
		p = np.sum(np.log(kf.c))
		print(f'The current iteration is: {ite}. The likelihood is {p}',end='\r')
		if p>p_old and p - p_old < tol:
			break
		p_old = p
		S_temp = np.matmul(np.matmul(kf.C, kf.V0), kf.C.T) + kf.Sigma
		Q_temp = np.matmul(kf.C, kf.u0)
		I = np.eye(kf.M)
		kf.V[0] = np.matmul((I - np.matmul(np.matmul(np.matmul(kf.V0, kf.C.T), np.linalg.inv(S_temp)), kf.C)), kf.V0)
		kf.P[0] = np.matmul(np.matmul(kf.A, kf.V[0]), kf.A.T) + kf.Gamma
		kf.K[0] = np.matmul(np.matmul(kf.P[0], kf.C.T), np.linalg.inv(np.matmul(np.matmul(kf.C, kf.P[0]), kf.C.T) + kf.Sigma))
        
		kf.u[0] = kf.u0 + np.matmul(kf.K[0], kf.x[0] - Q_temp)
		
	print('u0\n',kf.u0,'\nV0\n',kf.V0,'\nA\n',kf.A,'\nGamma\n',kf.Gamma,'\nC\n',kf.C,'\nSigma\n',kf.Sigma)
	return kf,z,x

kf,z,x = main()

/tmp/ipykernel_1384654/1217914384.py:54: RuntimeWarning: divide by zero encountered in log
  p = np.sum(np.log(kf.c))


u0e current iteration is: 156. The likelihood is -9420.379888278392
 [ 2.57600941  6.32556377 11.23378913] 
V0
 [[2.56102315e-05 2.46044142e-05 5.16388836e-05]
 [2.46044142e-05 1.08602395e-04 1.16694322e-04]
 [5.16388836e-05 1.16694322e-04 2.14669598e-04]] 
A
 [[ 0.78555585 -0.54696324  0.37224698]
 [ 0.89482765  0.872016   -0.11774004]
 [ 0.10568178  0.43474824  0.71648858]] 
Gamma
 [[0.00827258 0.02123041 0.03421632]
 [0.02123041 0.06677456 0.10594897]
 [0.03421632 0.10594897 0.17351357]] 
C
 [[-15.89162435  -3.13681034   9.65576876]
 [ 19.54139093   7.56212242  -4.38943598]] 
Sigma
 [[0.91199827 0.02540029]
 [0.02540029 1.89910543]]


In [ ]:
# def is_symmetric_positive_semidefinite(matrix):
#     if not np.allclose(matrix, matrix.T):
#         return False  # Not symmetric
#     eigenvalues = np.linalg.eigvals(matrix)
#     return np.all(eigenvalues >= 0)

# B = np.array([[35.62538664, 35.93218881],
#  [35.93218881, 36.40284164]])
# print(is_symmetric_positive_semidefinite(B)) 

In [ ]:
z_sim,x_sim = generate_examples(A = kf.A, C = kf.C, Gamma = kf.Gamma, Sigma = kf.Sigma, M=kf.M,N=kf.N,T=kf.T,u0=kf.u0,V0=kf.V0)


In [ ]:
print(x_sim[:10])

[[47.71947426 50.8485288 ]
 [41.2946439  53.57172734]
 [45.29068065 55.57419676]
 [46.98624198 56.02827182]
 [50.97759307 51.01487657]
 [53.8040345  48.18011965]
 [54.46937085 44.45606866]
 [55.18197221 47.6506192 ]
 [48.86497964 53.75402067]
 [43.67351015 51.46476942]]


In [ ]:
print(x[:10])

[[47.68902127 48.86610347]
 [43.94953287 54.53438865]
 [41.39095419 55.59938853]
 [45.94610067 55.73904685]
 [53.12714417 48.54030761]
 [58.62358006 46.04121809]
 [60.14091914 43.33480833]
 [50.83684644 42.69708909]
 [46.74694337 52.19143261]
 [44.63536853 55.59384609]]
